In [4]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
#
#
# Ugly workarround to increase spark.kryoserializer.buffer.max
# Only way in session to Run :
#        - treeAggregate at RowMatrix.scala:122 
#        - first at PCA.scala:43
#
sc = pyspark.SparkContext()
conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','2047mb'),('spark.driver.memory','23g'),('spark.driver.maxResultSize','21g')])
sc.stop()
sc = pyspark.SparkContext(conf=conf,appName="Notebook-FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-CleanData")
sqlContext = SQLContext(sc)
#
#
# Arguments
#
#import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date="20190122"
#
input_file1_playback_fraud="hdfs:///data/staged/ott_dazn/fraud-canada-tokenizedwords/dt=*/*.json"
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-canada-tokenizedwords-ngrams-85-90/dt="+process_date
#
input_file2_playback_not_fraud="hdfs:///data/staged/ott_dazn/logs-archive-production/parquet/dt="+process_date+"/*.parquet"
output_file2="hdfs:///data/staged/ott_dazn/advanced-model-data/not-fraud-canada-tokenizedwords/dt="+process_date
#
input_file3="hdfs:///data/staged/ott_dazn/advanced-model-data/not-fraud-canada-tokenizedwords/dt="+process_date+"/*.*"
output_file3="hdfs:///data/staged/ott_dazn/advanced-model-data/not-fraud-canada-tokenizedwords-ngrams-85-90/dt="+process_date
#
input_file4="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-canada-tokenizedwords-ngrams-85-90/dt="+process_date+"/*.*"
input_file5="hdfs:///data/staged/ott_dazn/not-fraud-canada-tokenizedwords-ngrams-85-90/dt="+process_date+"/*.*"
#
output_most_frequent_df="hdfs:///data/staged/ott_dazn/advanced-model-data/output_most_frequent_fraud_hash_message/dt="+process_date
#
# FILTER FRAUD AND LABEL 
# Join with Internal Curation Data in urltopredict staged folder
from pyspark.sql import functions as F
#
# hash the message de-duplicate those records
urlpredict_internalfraud_file=sqlContext.read.json(input_file1_playback_fraud)
urlpredict_internalfraud_file.printSchema()
#
urlpredict_internalfraud_df=urlpredict_internalfraud_file\
.filter("message IS NOT NULL").filter("words IS NOT NULL")\
.withColumn('fraud_label',lit(1).cast('int'))\
.withColumn('hash_message',F.sha2(col('message'),512)).groupby(col('hash_message'))\
.agg(F.first(col('fraud_label')).alias('fraud_label'),F.first(col('words')).alias('words'),F.first(col('message')).alias('message'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
urlpredict_internalfraud_df.printSchema()
#
df_words = urlpredict_internalfraud_df.filter("message IS NOT NULL").select(col('fraud_label'),col('hash_message'),col('words'))
#
## Register Generic Functions
# -----------------------------------------------------------------------------
# Build ngrams 85 90 n=6 
# support : https://stackoverflow.com/questions/51473703/pyspark-ml-ngrams-countvectorizer-sorted-based-on-count-weights
# -----------------------------------------------------------------------------
def build_ngrams_part(inputCol="words", n=6):
    ngrams = [ 
        NGram(n=i, inputCol="words", outputCol="ngrams_{0}".format(i)) 
        for i in range(85, n + 1) ]
    vectorizers = [ 
        CountVectorizer(inputCol="ngrams_{0}".format(i), outputCol="ngramscounts_{0}".format(i)) 
        for i in range(85, n + 1) ]
    return Pipeline(stages=ngrams + vectorizers)
#    assembler = [VectorAssembler( inputCols=["ngramscounts_{0}".format(i) for i in range(1, n + 1)], outputCol="features" )]
#    return Pipeline(stages=ngrams + DenseVector(SparseVector(vectorizers).toArray()))
#
# 
# -----------------------------------------------------------------------------
#ngram = build_ngrams_part().fit(df_words)
#ngramDataFrame = ngram.transform(df_words)
#ngramDataFrame.coalesce(1).write.json(output_file1)
#
ngram = NGram(n=85, inputCol="words", outputCol="ngrams_85")
countvector = CountVectorizer(inputCol="ngrams_85", outputCol="ngramscounts_85")
# fit a CountVectorizerModel from the corpus.
countvModel = CountVectorizer(inputCol="words", outputCol="features_85", vocabSize=85, minDF=2.0)
# fit a PCA Dimensionality reduction into 6/2=3 components from ngramscounts_85 ## Too Heavy 1st PCA
pcaNgrams = PCA(k=3, inputCol="ngramscounts_85", outputCol="pcaweightngrams")
# fit a PCA Dimensionality reduction into 85/17=5 components from words
pcaWords = PCA(k=5, inputCol="features_85", outputCol="pcaweightwords")  ## Too Heavy 2nd PCA
#
ngram_fraud_DF = ngram.transform(df_words)
ngram_vc_fraud_DF = countvector.fit(ngram_fraud_DF).transform(ngram_fraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
ngram_vc_fraud_DF.printSchema()
#
#modelPCA_ngram_fraud_DF = pcaNgrams.fit(ngram_vc_fraud_DF).transform(ngram_vc_fraud_DF)\
#.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#modelPCA_ngram_fraud_DF.printSchema()
#
result_ngrams_words_fraud_DF = countvModel.fit(ngram_vc_fraud_DF).transform(ngram_vc_fraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
result_ngrams_words_fraud_DF.printSchema()
#
result_ngrams_words_fraud_DF.coalesce(1).write.json(output_file1)
#
#  FILTER Non-Fraud AND LABEL
#
df2= sqlContext.read.parquet(input_file2_playback_not_fraud)
df2.printSchema()
#
df3 = df2.filter(" (message LIKE '%\"Url\":\"https://isl-ca.dazn.com/misl/v2/Playback%') AND (message LIKE '%,\"Response\":{\"StatusCode\":200,\"ReasonPhrase\":\"OK\",%') AND ( ( (message LIKE '%&Format=MPEG-DASH&%' OR message LIKE '%&Format=M3U&%') ) OR (message NOT LIKE '%\"User-Agent\":\"Mozilla/5.0,(Macintosh; Intel Mac OS X 10_12_6),AppleWebKit/605.1.15,(KHTML, like Gecko),Version/11.1.2,Safari/605.1.15\"},%')   )  ")
df3.printSchema()
df4 = df3.withColumn("messagecut", expr("substring(message, locate('|Livesport.WebApi.Controllers.Playback.PlaybackV2Controller|',message)+60 , length(message)-1)"))
#
# val regexTokenizer = new RegexTokenizer().setInputCol("messagecut").setOutputCol("words").setPattern("\\w+|").setGaps(false)
#
regexTokenizer = RegexTokenizer(minTokenLength=1, gaps=False, pattern='\\w+|', inputCol="messagecut", outputCol="words", toLowercase=True)
#
tokenized = regexTokenizer.transform(df4)
tokenized.printSchema()
tokenized.coalesce(1).write.json(output_file2)
# Tokenize NON-Fraud-LABEL
# hash the message de-duplicate those records
notfraud_file=sqlContext.read.json(input_file3)
notfraud_file.printSchema()
#
notfraud_df=notfraud_file\
.filter("message IS NOT NULL").filter("words IS NOT NULL")\
.withColumn('fraud_label',lit(0).cast('int'))\
.withColumn('hash_message',F.sha2(col('message'),512)).groupby(col('hash_message'))\
.agg(F.first(col('fraud_label')).alias('fraud_label'),F.first(col('words')).alias('words'),F.first(col('message')).alias('message'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
notfraud_df.printSchema()
#
df_notfraud_words = notfraud_df.filter("message IS NOT NULL").select(col('fraud_label'),col('hash_message'),col('words'))
#
##
## Register Generic Functions
# -----------------------------------------------------------------------------
# Build ngrams 85 90 n=6 
# support : https://stackoverflow.com/questions/51473703/pyspark-ml-ngrams-countvectorizer-sorted-based-on-count-weights
# -----------------------------------------------------------------------------
def build_ngrams_part(inputCol="words", n=6):
    ngrams = [ 
        NGram(n=i, inputCol="words", outputCol="ngrams_{0}".format(i)) 
        for i in range(85, n + 1) ]
    vectorizers = [ 
        CountVectorizer(inputCol="ngrams_{0}".format(i), outputCol="ngramscounts_{0}".format(i)) 
        for i in range(85, n + 1) ]
    return Pipeline(stages=ngrams + vectorizers)
#    assembler = [VectorAssembler(inputCols=["ngramscounts_{0}".format(i) for i in range(1, n + 1)], outputCol="features" )]
#    return Pipeline(stages=ngrams + DenseVector(SparseVector(vectorizers).toArray()))
#
#
# 
# -----------------------------------------------------------------------------
#ngram = build_ngrams_part().fit(df_notfraud_words)
#ngram_notfraud_DataFrame = ngram.transform(df_notfraud_words)
#ngram_notfraud_DataFrame.coalesce(1).write.json(output_file3)
#
ngram = NGram(n=85, inputCol="words", outputCol="ngrams_85")
countvector = CountVectorizer(inputCol="ngrams_85", outputCol="ngramscounts_85")
# fit a CountVectorizerModel from the corpus.
countvModel = CountVectorizer(inputCol="words", outputCol="features_85", vocabSize=85, minDF=2.0)
# fit a PCA Dimensionality reduction into 6/2=3 components from ngramscounts_85 ## Too Heavy 1st PCA
pcaNgrams = PCA(k=3, inputCol="ngramscounts_85", outputCol="pcaweightngrams")
# fit a PCA Dimensionality reduction into 85/17=5 components from words ## Too Heavy 2nd PCA
pcaWords = PCA(k=5, inputCol="features_85", outputCol="pcaweightwords")
#
ngram_notfraud_DF = ngram.transform(df_notfraud_words)
ngram_vc_notfraud_DF = countvector.fit(ngram_notfraud_DF).transform(ngram_notfraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
ngram_vc_notfraud_DF.printSchema()
#
#modelPCA_ngram_notfraud_DF = pcaNgrams.fit(ngram_vc_notfraud_DF).transform(ngram_notfraud_DF)\
#.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#modelPCA_ngram_notfraud_DF.printSchema()
#
result_ngrams_words_notfraud_DF = countvModel.fit(ngram_vc_notfraud_DF).transform(ngram_vc_notfraud_DF)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
result_ngrams_words_notfraud_DF.printSchema()
#
result_ngrams_words_notfraud_DF.coalesce(1).write.json(output_file3)
#
#  CALCULATE KL,KS COEF. Label Data
ngram8590_fraud=sqlContext.read.json(input_file4)
ngram8590_fraud.printSchema()
#
print("Calculation of standard_fraud_ngram - Start!")
#
# Obtain the most frequent word on each position 
# Compose the standard_fraud_ngram from the most common positions
# Calculate the standard_fraud_ngram
#
# https://stackoverflow.com/questions/35218882/find-maximum-row-per-group-in-spark-dataframe 
# Using struct ordering:
#from pyspark.sql.functions import struct
#
#(cnts
#  .groupBy("id_sa")
#  .agg(F.max(struct(col("cnt"), col("id_sb"))).alias("max"))
#  .select(col("id_sa"), col("max.id_sb")))
#
#Py4JJavaError: An error occurred while calling o3400.select.
#: org.apache.spark.sql.AnalysisException: cannot resolve '`ngrams`' given input columns: [mostfrequent];;
#'Project ['ngrams, mostfrequent#8033.count AS count#8037L]
#+- AnalysisBarrier
#      +- Aggregate [max(named_struct(count, count#8029L)) AS mostfrequent#8033]
#         +- Aggregate [ngrams#8016], [ngrams#8016, count(1) AS count#8029L]
#            +- Relation[fraud_label#8013L,hash_message#8014,message#8015,ngrams#8016,words#8017] json
#
####### OLD  ######
#.groupby(col('hash_message'))\
#.agg(F.first(col('features_85')).alias('features_85'),\
#     F.first(col('fraud_label')).alias('fraud_label'),\
#     F.first(col('ngrams_85')).alias('ngrams_85'),\
#     F.first(col('ngramscounts_85')).alias('ngramscounts_85'),F.first(col('words')).alias('words'))
####################
#
most_frequent_df=ngram8590_fraud\
.withColumn("value_sum",F.explode("ngramscounts_85.values"))\
.groupBy("hash_message").agg(F.sum("value_sum").alias('count'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
most_frequent_df.printSchema()
#
most_frequent_df.coalesce(1).write.json(output_most_frequent_df)
#
# The most Frequent would the the max
standard_fraud_ngram=most_frequent_df.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()
#
print("Value Print: standard_fraud_ngram=")
print(standard_fraud_ngram)
#
print("Calculation of standard_fraud_ngram - Done!")
#
sc.stop()
#
print("Preparation of Data Done!")
#

root
 |-- @metadata: struct (nullable = true)
 |    |-- beat: string (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- @timestamp: string (nullable = true)
 |-- __logz_account_id: string (nullable = true)
 |-- __logz_received_timestamp: string (nullable = true)
 |-- __logzio_X-B3-Sampled: string (nullable = true)
 |-- __logzio_X-B3-SpanId: string (nullable = true)
 |-- __logzio_X-B3-TraceId: string (nullable = true)
 |-- __logzio_id: string (nullable = true)
 |-- beat: struct (nullable = true)
 |    |-- hostname: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- version: string (nullable = true)
 |-- host: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |-- input_type: string (nullable = true)
 |-- it: string (nullable = true)
 |-- logzio_codec: string (nullable = true)
 |-- logzio_id: string (nullable = true)
 |-- message: string (nullable = true)
 |-- messagecut: string (nullable = 

root
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- ngrams_85: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngramscounts_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)

Calculation of standard_fraud_ngram - Start!
root
 |-- hash_message: string (nullable = true)
 |-- count: double 

In [5]:
sc.stop()

In [ ]:
#
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
#
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import RegexTokenizer
#
#import org.apache.spark.ml.feature.NGram
from pyspark.ml.feature import NGram
#
from collections import Counter
#
from pyspark.ml.feature import NGram
#
from pyspark.ml.feature import NGram, CountVectorizer, VectorAssembler
from pyspark.ml import Pipeline
#
from pyspark.mllib.linalg import SparseVector, DenseVector
#
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
#
# FUNCTIONS
# 
##
## Register Generic UDF Functions
# -----------------------------------------------------------------------------
# KL : Kullback-Leibler Divergence
# KS : Kolmogorov-Smirnov ( Sample sizes can be different)
# -----------------------------------------------------------------------------    
## NGRAM KL from URL TO Default NGRAM KL
def func_kl_ngram_msg(var1,var_match):
    def KL(P,Q):
        epsilon = 0.00001
        P = P+epsilon
        Q = Q+epsilon
        divergence = np.sum(P*np.log(P/Q))
        return np.asscalar(divergence)
    ##Making sure the analysis ignore uri and focus on typical message site names.
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar.split("/")[0]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    # URLS with at least 4166 characters
    ## TODO : Complete this list with common non-Malware/phishing sites used in internal webtraffic
    list_values2 = var_match
    ## Making sure the analysis start on the same webdomain name
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    print(list_values1,list_values2)
    print(values1,values2)
    return KL(values1,values2)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
## NGRAM "Kolmogorov-Smirnov test KS pvalue= ks_2samp function8.
def func_ks_ngram_msg(var1,var_match):
    ##Making sure the analysis ignore uri and focus on typical message names.
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar.split("/")[0]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    values1= np.sort(values1)
    # URLS with at least 4166 characters
    ## TODO : Complete this list with common non-Malware/phishing sites used in internal webtraffic
    list_values2 = var_match
    # Making sure the analysis start on the same webdomain name
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    values2=np.sort(values2)
    (Darray,pvalue)=stats.ks_2samp(values1, values2)
    return np.asscalar(pvalue)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
## URL Entropy TO phishing URL function17.
def func_entropy_msg(var1,var2):
    ##Making sure the analysis ignore uri and focus on typical message names.
    cleanvar=var1.strip('http://').strip('https://').strip('www.')
    len_var1=len(cleanvar)
    idx_var1=cleanvar.split("/")[0]
    list_values1 = list(cleanvar)
    list_of_ord_values1 = [ord(char) for char in list_values1]
    values1 = np.asarray(list_of_ord_values1)
    # URLS with Up to 4166 characters
    list_values2 = var2
    ## Making sure the analysis start on the same webdomain name
    idx_var2 = list_values2.find(idx_var1)
    start_values2 = list_values2[idx_var2:(idx_var2+len_var1)]
    list_values2 = list(start_values2+list_values2)
    list_values2 = list_values2[:len_var1]
    list_of_ord_values2 = [ord(char) for char in list_values2]
    values2 = np.asarray(list_of_ord_values2)
    pvalue=stats.entropy(values1, values2)
    scalar_pvalue=np.asscalar(pvalue)
    return scalar_pvalue
#
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
func_kl_ngram_msg_udf = udf(func_kl_ngram_msg, FloatType())
func_ks_ngram_msg_udf = udf(func_ks_ngram_msg, FloatType())
func_entropy_msg_udf = udf(func_entropy_msg, FloatType())
#
# Arguments
#
import argparse
## Parse date_of execution
#parser = argparse.ArgumentParser()
#parser.add_argument("--datev1", help="Execution Date")
#args = parser.parse_args()
#if args.datev1:
#    processdate = args.datev1
# GENERAL PREPARATION SCRIPT
#
#  Date in format YYYYMMDD
#process_date = processdate
#if not process_date:
#    process_date = "20181231"
#
process_date="20190122"
#
#
sc = pyspark.SparkContext()
conf = pyspark.SparkConf().setAll([('spark.kryoserializer.buffer.max','2047mb'),('spark.driver.memory','23g'),('spark.driver.maxResultSize','21g')])
sc.stop()
sc = pyspark.SparkContext(conf=conf,appName="Notebook-FraudCanada-Model-NGrams-CountVectorizer-KL-KS-Entropy-Model-Discovery")
sqlContext = SQLContext(sc)
#
import h2o
from h2o.automl import H2OAutoML
#
import subprocess
subprocess.run('unset http_proxy', shell=True)
#
# Start an H2O virtual cluster that uses 6 gigs of RAM and 6 cores
h2o.init(ip="localhost",port=54321,max_mem_size = "6g", nthreads = 6) 
#
# Clean up the cluster just in case
h2o.remove_all()
#
input_most_frequent_df="hdfs:///data/staged/ott_dazn/advanced-model-data/output_most_frequent_fraud_hash_message/dt="+process_date
#
input_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/fraud-canada-tokenizedwords-ngrams-85-90/dt="+process_date
output_file1="hdfs:///data/staged/ott_dazn/advanced-model-data/label-fraud-data-model/dt="+process_date
#
input_file2="hdfs:///data/staged/ott_dazn/advanced-model-data/not-fraud-canada-tokenizedwords-ngrams-85-90/dt="+process_date
output_file2="hdfs:///data/staged/ott_dazn/advanced-model-data/label-not-fraud-data-model/dt="+process_date
#
#
# The most Frequent would the the max
pd.options.display.max_colwidth = 512
standard_fraud_ngram=sqlContext.read.json(input_most_frequent_df)\
.orderBy(col('count').desc()).select(col('hash_message')).limit(1).toPandas()['hash_message'][0]
print("Value UDF : standard_fraud_ngram=")
print(standard_fraud_ngram)
#
#
ngram8590_fraud=sqlContext.read.json(input_file1)
ngram8590_fraud.printSchema()
#
drop_phish_cols=['message']
#
fraud_label_read_df=ngram8590_fraud\
.filter("hash_message is not NULL")\
.drop(*drop_phish_cols)\
.withColumn('kl_hash_message',func_kl_ngram_msg_udf(col('hash_message'),lit(standard_fraud_ngram).cast('string') ).cast('double'))\
.withColumn('ks_hash_message',func_ks_ngram_msg_udf(col('hash_message'),lit(standard_fraud_ngram).cast('string')).cast('double'))\
.withColumn('entropy_hash_message',func_entropy_msg_udf(col('hash_message'),lit(standard_fraud_ngram).cast('string')).cast('double'))
#
fraud_label_read_df.coalesce(1).write.json(output_file1)
#
ngram8590_not_fraud=sqlContext.read.json(input_file2)
ngram8590_not_fraud.printSchema()
#
not_fraud_label_read_df=ngram8590_not_fraud\
.filter("hash_message is not NULL")\
.drop(*drop_phish_cols)\
.withColumn('kl_hash_message',func_kl_ngram_msg_udf(col('hash_message'),lit(standard_fraud_ngram).cast('string')).cast('double'))\
.withColumn('ks_hash_message',func_ks_ngram_msg_udf(col('hash_message'),lit(standard_fraud_ngram).cast('string') ).cast('double'))\
.withColumn('entropy_hash_message',func_entropy_msg_udf(col('hash_message'),lit(standard_fraud_ngram).cast('string')).cast('double'))
#
not_fraud_label_read_df.coalesce(1).write.json(output_file2)
#
#
#  TRAINING PROCESS
#
print("Start Training Model NGrams Vectors KS KL Entropty")
#
# Horrible code :: close your eyes, is ugly
#
fraud_label_read_df=sqlContext.read.json(output_file1).\
select(col('hash_message').cast('string'), col('fraud_label').cast('int'),\
       col('kl_hash_message').cast('double'), col('ks_hash_message').cast('double'),\
       col('entropy_hash_message').cast('double'), col('features_85.size').cast('int'),\
       col('features_85.type').cast('int'),\
       substring_index(col('features_85.indexes'),',',0).cast('double').alias('features_85_indexes_1'),\
       substring_index(col('features_85.indexes'),',',1).cast('double').alias('features_85_indexes_2'),\
       substring_index(col('features_85.indexes'),',',2).cast('double').alias('features_85_indexes_4'),\
       substring_index(col('features_85.indexes'),',',3).cast('double').alias('features_85_indexes_4'),\
       substring_index(col('features_85.indexes'),',',4).cast('double').alias('features_85_indexes_5'),\
       substring_index(col('features_85.indexes'),',',5).cast('double').alias('features_85_indexes_6'),\
       substring_index(col('features_85.indexes'),',',6).cast('double').alias('features_85_indexes_7'),\
       substring_index(col('features_85.indexes'),',',7).cast('double').alias('features_85_indexes_8'),\
       substring_index(col('features_85.indexes'),',',8).cast('double').alias('features_85_indexes_9'),\
       substring_index(col('features_85.indexes'),',',9).cast('double').alias('features_85_indexes_10'),\
       substring_index(col('features_85.indexes'),',',10).cast('double').alias('features_85_indexes_11'),\
       substring_index(col('features_85.indexes'),',',11).cast('double').alias('features_85_indexes_12'),\
       substring_index(col('features_85.indexes'),',',12).cast('double').alias('features_85_indexes_13'),\
       substring_index(col('features_85.indexes'),',',13).cast('double').alias('features_85_indexes_14'),\
       substring_index(col('features_85.indexes'),',',14).cast('double').alias('features_85_indexes_15'),\
       substring_index(col('features_85.indexes'),',',15).cast('double').alias('features_85_indexes_16'),\
       substring_index(col('features_85.indexes'),',',16).cast('double').alias('features_85_indexes_17'),\
       substring_index(col('features_85.indexes'),',',17).cast('double').alias('features_85_indexes_18'),\
       substring_index(col('features_85.indexes'),',',18).cast('double').alias('features_85_indexes_19'),\
       substring_index(col('features_85.indexes'),',',19).cast('double').alias('features_85_indexes_20'),\
       substring_index(col('features_85.indexes'),',',20).cast('double').alias('features_85_indexes_21'),\
       substring_index(col('features_85.indexes'),',',21).cast('double').alias('features_85_indexes_22'),\
       substring_index(col('features_85.indexes'),',',22).cast('double').alias('features_85_indexes_23'),\
       substring_index(col('features_85.indexes'),',',23).cast('double').alias('features_85_indexes_24'),\
       substring_index(col('features_85.indexes'),',',24).cast('double').alias('features_85_indexes_25'),\
       substring_index(col('features_85.indexes'),',',25).cast('double').alias('features_85_indexes_26'),\
       substring_index(col('features_85.indexes'),',',26).cast('double').alias('features_85_indexes_27'),\
       substring_index(col('features_85.indexes'),',',27).cast('double').alias('features_85_indexes_28'),\
       substring_index(col('features_85.indexes'),',',28).cast('double').alias('features_85_indexes_29'),\
       substring_index(col('features_85.indexes'),',',29).cast('double').alias('features_85_indexes_30'),\
       substring_index(col('features_85.indexes'),',',30).cast('double').alias('features_85_indexes_31'),\
       substring_index(col('features_85.indexes'),',',31).cast('double').alias('features_85_indexes_32'),\
       substring_index(col('features_85.indexes'),',',32).cast('double').alias('features_85_indexes_33'),\
       substring_index(col('features_85.indexes'),',',33).cast('double').alias('features_85_indexes_34'),\
       substring_index(col('features_85.indexes'),',',34).cast('double').alias('features_85_indexes_35'),\
       substring_index(col('features_85.indexes'),',',35).cast('double').alias('features_85_indexes_36'),\
       substring_index(col('features_85.indexes'),',',36).cast('double').alias('features_85_indexes_37'),\
       substring_index(col('features_85.indexes'),',',37).cast('double').alias('features_85_indexes_38'),\
       substring_index(col('features_85.indexes'),',',38).cast('double').alias('features_85_indexes_39'),\
       substring_index(col('features_85.indexes'),',',39).cast('double').alias('features_85_indexes_40'),\
       substring_index(col('features_85.indexes'),',',40).cast('double').alias('features_85_indexes_41'),\
       substring_index(col('features_85.indexes'),',',41).cast('double').alias('features_85_indexes_42'),\
       substring_index(col('features_85.indexes'),',',42).cast('double').alias('features_85_indexes_43'),\
       substring_index(col('features_85.indexes'),',',43).cast('double').alias('features_85_indexes_44'),\
       substring_index(col('features_85.indexes'),',',44).cast('double').alias('features_85_indexes_45'),\
       substring_index(col('features_85.indexes'),',',45).cast('double').alias('features_85_indexes_46'),\
       substring_index(col('features_85.indexes'),',',46).cast('double').alias('features_85_indexes_47'),\
       substring_index(col('features_85.indexes'),',',47).cast('double').alias('features_85_indexes_48'),\
       substring_index(col('features_85.indexes'),',',48).cast('double').alias('features_85_indexes_49'),\
       substring_index(col('features_85.indexes'),',',49).cast('double').alias('features_85_indexes_50'),\
       substring_index(col('features_85.indexes'),',',50).cast('double').alias('features_85_indexes_51'),\
       substring_index(col('features_85.indexes'),',',51).cast('double').alias('features_85_indexes_52'),\
       substring_index(col('features_85.indexes'),',',52).cast('double').alias('features_85_indexes_53'),\
       substring_index(col('features_85.indexes'),',',53).cast('double').alias('features_85_indexes_54'),\
       substring_index(col('features_85.indexes'),',',54).cast('double').alias('features_85_indexes_55'),\
       substring_index(col('features_85.indexes'),',',55).cast('double').alias('features_85_indexes_56'),\
       substring_index(col('features_85.indexes'),',',56).cast('double').alias('features_85_indexes_57'),\
       substring_index(col('features_85.indexes'),',',57).cast('double').alias('features_85_indexes_58'),\
       substring_index(col('features_85.indexes'),',',58).cast('double').alias('features_85_indexes_59'),\
       substring_index(col('features_85.indexes'),',',59).cast('double').alias('features_85_indexes_60'),\
       substring_index(col('features_85.indexes'),',',60).cast('double').alias('features_85_indexes_61'),\
       substring_index(col('features_85.indexes'),',',61).cast('double').alias('features_85_indexes_62'),\
       substring_index(col('features_85.indexes'),',',62).cast('double').alias('features_85_indexes_63'),\
       substring_index(col('features_85.indexes'),',',63).cast('double').alias('features_85_indexes_64'),\
       substring_index(col('features_85.indexes'),',',64).cast('double').alias('features_85_indexes_65'),\
       substring_index(col('features_85.indexes'),',',65).cast('double').alias('features_85_indexes_66'),\
       substring_index(col('features_85.indexes'),',',66).cast('double').alias('features_85_indexes_67'),\
       substring_index(col('features_85.indexes'),',',67).cast('double').alias('features_85_indexes_68'),\
       substring_index(col('features_85.indexes'),',',68).cast('double').alias('features_85_indexes_69'),\
       substring_index(col('features_85.indexes'),',',69).cast('double').alias('features_85_indexes_70'),\
       substring_index(col('features_85.indexes'),',',70).cast('double').alias('features_85_indexes_71'),\
       substring_index(col('features_85.indexes'),',',71).cast('double').alias('features_85_indexes_72'),\
       substring_index(col('features_85.indexes'),',',72).cast('double').alias('features_85_indexes_73'),\
       substring_index(col('features_85.indexes'),',',73).cast('double').alias('features_85_indexes_74'),\
       substring_index(col('features_85.indexes'),',',74).cast('double').alias('features_85_indexes_75'),\
       substring_index(col('features_85.indexes'),',',75).cast('double').alias('features_85_indexes_76'),\
       substring_index(col('features_85.indexes'),',',76).cast('double').alias('features_85_indexes_77'),\
       substring_index(col('features_85.indexes'),',',77).cast('double').alias('features_85_indexes_78'),\
       substring_index(col('features_85.indexes'),',',78).cast('double').alias('features_85_indexes_70'),\
       substring_index(col('features_85.indexes'),',',79).cast('double').alias('features_85_indexes_80'),\
       substring_index(col('features_85.indexes'),',',80).cast('double').alias('features_85_indexes_82'),\
       substring_index(col('features_85.indexes'),',',81).cast('double').alias('features_85_indexes_82'),\
       substring_index(col('features_85.indexes'),',',82).cast('double').alias('features_85_indexes_83'),\
       substring_index(col('features_85.indexes'),',',83).cast('double').alias('features_85_indexes_84'),\
       substring_index(col('features_85.indexes'),',',84).cast('double').alias('features_85_indexes_85'),\
       substring_index(col('features_85.values'),',',0).cast('double').alias('features_85_values_1'),\
       substring_index(col('features_85.values'),',',0).cast('double').alias('features_85_values_1'),\
       substring_index(col('features_85.values'),',',1).cast('double').alias('features_85_values_2'),\
       substring_index(col('features_85.values'),',',2).cast('double').alias('features_85_values_4'),\
       substring_index(col('features_85.values'),',',3).cast('double').alias('features_85_values_4'),\
       substring_index(col('features_85.values'),',',4).cast('double').alias('features_85_values_5'),\
       substring_index(col('features_85.values'),',',5).cast('double').alias('features_85_values_6'),\
       substring_index(col('features_85.values'),',',6).cast('double').alias('features_85_values_7'),\
       substring_index(col('features_85.values'),',',7).cast('double').alias('features_85_values_8'),\
       substring_index(col('features_85.values'),',',8).cast('double').alias('features_85_values_9'),\
       substring_index(col('features_85.values'),',',9).cast('double').alias('features_85_values_10'),\
       substring_index(col('features_85.values'),',',10).cast('double').alias('features_85_values_11'),\
       substring_index(col('features_85.values'),',',11).cast('double').alias('features_85_values_12'),\
       substring_index(col('features_85.values'),',',12).cast('double').alias('features_85_values_13'),\
       substring_index(col('features_85.values'),',',13).cast('double').alias('features_85_values_14'),\
       substring_index(col('features_85.values'),',',14).cast('double').alias('features_85_values_15'),\
       substring_index(col('features_85.values'),',',15).cast('double').alias('features_85_values_16'),\
       substring_index(col('features_85.values'),',',16).cast('double').alias('features_85_values_17'),\
       substring_index(col('features_85.values'),',',17).cast('double').alias('features_85_values_18'),\
       substring_index(col('features_85.values'),',',18).cast('double').alias('features_85_values_19'),\
       substring_index(col('features_85.values'),',',19).cast('double').alias('features_85_values_20'),\
       substring_index(col('features_85.values'),',',20).cast('double').alias('features_85_values_21'),\
       substring_index(col('features_85.values'),',',21).cast('double').alias('features_85_values_22'),\
       substring_index(col('features_85.values'),',',22).cast('double').alias('features_85_values_23'),\
       substring_index(col('features_85.values'),',',23).cast('double').alias('features_85_values_24'),\
       substring_index(col('features_85.values'),',',24).cast('double').alias('features_85_values_25'),\
       substring_index(col('features_85.values'),',',25).cast('double').alias('features_85_values_26'),\
       substring_index(col('features_85.values'),',',26).cast('double').alias('features_85_values_27'),\
       substring_index(col('features_85.values'),',',27).cast('double').alias('features_85_values_28'),\
       substring_index(col('features_85.values'),',',28).cast('double').alias('features_85_values_29'),\
       substring_index(col('features_85.values'),',',29).cast('double').alias('features_85_values_30'),\
       substring_index(col('features_85.values'),',',30).cast('double').alias('features_85_values_31'),\
       substring_index(col('features_85.values'),',',31).cast('double').alias('features_85_values_32'),\
       substring_index(col('features_85.values'),',',32).cast('double').alias('features_85_values_33'),\
       substring_index(col('features_85.values'),',',33).cast('double').alias('features_85_values_34'),\
       substring_index(col('features_85.values'),',',34).cast('double').alias('features_85_values_35'),\
       substring_index(col('features_85.values'),',',35).cast('double').alias('features_85_values_36'),\
       substring_index(col('features_85.values'),',',36).cast('double').alias('features_85_values_37'),\
       substring_index(col('features_85.values'),',',37).cast('double').alias('features_85_values_38'),\
       substring_index(col('features_85.values'),',',38).cast('double').alias('features_85_values_39'),\
       substring_index(col('features_85.values'),',',39).cast('double').alias('features_85_values_40'),\
       substring_index(col('features_85.values'),',',40).cast('double').alias('features_85_values_41'),\
       substring_index(col('features_85.values'),',',41).cast('double').alias('features_85_values_42'),\
       substring_index(col('features_85.values'),',',42).cast('double').alias('features_85_values_43'),\
       substring_index(col('features_85.values'),',',43).cast('double').alias('features_85_values_44'),\
       substring_index(col('features_85.values'),',',44).cast('double').alias('features_85_values_45'),\
       substring_index(col('features_85.values'),',',45).cast('double').alias('features_85_values_46'),\
       substring_index(col('features_85.values'),',',46).cast('double').alias('features_85_values_47'),\
       substring_index(col('features_85.values'),',',47).cast('double').alias('features_85_values_48'),\
       substring_index(col('features_85.values'),',',48).cast('double').alias('features_85_values_49'),\
       substring_index(col('features_85.values'),',',49).cast('double').alias('features_85_values_50'),\
       substring_index(col('features_85.values'),',',50).cast('double').alias('features_85_values_51'),\
       substring_index(col('features_85.values'),',',51).cast('double').alias('features_85_values_52'),\
       substring_index(col('features_85.values'),',',52).cast('double').alias('features_85_values_53'),\
       substring_index(col('features_85.values'),',',53).cast('double').alias('features_85_values_54'),\
       substring_index(col('features_85.values'),',',54).cast('double').alias('features_85_values_55'),\
       substring_index(col('features_85.values'),',',55).cast('double').alias('features_85_values_56'),\
       substring_index(col('features_85.values'),',',56).cast('double').alias('features_85_values_57'),\
       substring_index(col('features_85.values'),',',57).cast('double').alias('features_85_values_58'),\
       substring_index(col('features_85.values'),',',58).cast('double').alias('features_85_values_59'),\
       substring_index(col('features_85.values'),',',59).cast('double').alias('features_85_values_60'),\
       substring_index(col('features_85.values'),',',60).cast('double').alias('features_85_values_61'),\
       substring_index(col('features_85.values'),',',61).cast('double').alias('features_85_values_62'),\
       substring_index(col('features_85.values'),',',62).cast('double').alias('features_85_values_63'),\
       substring_index(col('features_85.values'),',',63).cast('double').alias('features_85_values_64'),\
       substring_index(col('features_85.values'),',',64).cast('double').alias('features_85_values_65'),\
       substring_index(col('features_85.values'),',',65).cast('double').alias('features_85_values_66'),\
       substring_index(col('features_85.values'),',',66).cast('double').alias('features_85_values_67'),\
       substring_index(col('features_85.values'),',',67).cast('double').alias('features_85_values_68'),\
       substring_index(col('features_85.values'),',',68).cast('double').alias('features_85_values_69'),\
       substring_index(col('features_85.values'),',',69).cast('double').alias('features_85_values_70'),\
       substring_index(col('features_85.values'),',',70).cast('double').alias('features_85_values_71'),\
       substring_index(col('features_85.values'),',',71).cast('double').alias('features_85_values_72'),\
       substring_index(col('features_85.values'),',',72).cast('double').alias('features_85_values_73'),\
       substring_index(col('features_85.values'),',',73).cast('double').alias('features_85_values_74'),\
       substring_index(col('features_85.values'),',',74).cast('double').alias('features_85_values_75'),\
       substring_index(col('features_85.values'),',',75).cast('double').alias('features_85_values_76'),\
       substring_index(col('features_85.values'),',',76).cast('double').alias('features_85_values_77'),\
       substring_index(col('features_85.values'),',',77).cast('double').alias('features_85_values_78'),\
       substring_index(col('features_85.values'),',',78).cast('double').alias('features_85_values_70'),\
       substring_index(col('features_85.values'),',',79).cast('double').alias('features_85_values_80'),\
       substring_index(col('features_85.values'),',',80).cast('double').alias('features_85_values_82'),\
       substring_index(col('features_85.values'),',',81).cast('double').alias('features_85_values_82'),\
       substring_index(col('features_85.values'),',',82).cast('double').alias('features_85_values_83'),\
       substring_index(col('features_85.values'),',',83).cast('double').alias('features_85_values_84'),\
       substring_index(col('features_85.values'),',',84).cast('double').alias('features_85_values_85'),\
       col('ngramscounts_85.size').cast('int'), col('ngramscounts_85.type').cast('int'),\
       substring_index(col('ngramscounts_85.indexes'),',',0).cast('double').alias('ngramscounts_85_indexes_1'),\
       substring_index(col('ngramscounts_85.indexes'),',',1).cast('double').alias('ngramscounts_85_indexes_2'),\
       substring_index(col('ngramscounts_85.indexes'),',',2).cast('double').alias('ngramscounts_85_indexes_3'),\
       substring_index(col('ngramscounts_85.indexes'),',',3).cast('double').alias('ngramscounts_85_indexes_4'),\
       substring_index(col('ngramscounts_85.indexes'),',',4).cast('double').alias('ngramscounts_85_indexes_5'),\
       substring_index(col('ngramscounts_85.indexes'),',',5).cast('double').alias('ngramscounts_85_indexes_6'),\
       substring_index(col('ngramscounts_85.indexes'),',',6).cast('double').alias('ngramscounts_85_indexes_7'),\
       substring_index(col('ngramscounts_85.indexes'),',',7).cast('double').alias('ngramscounts_85_indexes_8'),\
       substring_index(col('ngramscounts_85.values'),',',0).cast('double').alias('ngramscounts_85values_1'),\
       substring_index(col('ngramscounts_85.values'),',',1).cast('double').alias('ngramscounts_85values_2'),\
       substring_index(col('ngramscounts_85.values'),',',2).cast('double').alias('ngramscounts_85values_3'),\
       substring_index(col('ngramscounts_85.values'),',',3).cast('double').alias('ngramscounts_85values_4'),\
       substring_index(col('ngramscounts_85.values'),',',4).cast('double').alias('ngramscounts_85values_5'),\
       substring_index(col('ngramscounts_85.values'),',',5).cast('double').alias('ngramscounts_85values_6'),\
       substring_index(col('ngramscounts_85.values'),',',6).cast('double').alias('ngramscounts_85values_7'),\
       substring_index(col('ngramscounts_85.values'),',',7).cast('double').alias('ngramscounts_85values_8'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
fraud_label_train_pd=fraud_label_read_df.toPandas()
fraud_label_test_pd=fraud_label_train_pd.toPandas().tail(500)                                                                        
fraud_label_train=h2o.H2OFrame(fraud_label_train_pd)
fraud_label_test=h2o.H2OFrame(fraud_label_test_pd)
#
# Horrible code :: close your eyes, is ugly
#
not_fraud_label_read_df=sqlContext.read.json(output_file2).\
select(col('hash_message').cast('string'), col('fraud_label').cast('int'),\
       col('kl_hash_message').cast('double'), col('ks_hash_message').cast('double'),\
       col('entropy_hash_message').cast('double'), col('features_85.size').cast('int'),\
       col('features_85.type').cast('int'),\
       substring_index(col('features_85.indexes'),',',0).cast('double').alias('features_85_indexes_1'),\
       substring_index(col('features_85.indexes'),',',1).cast('double').alias('features_85_indexes_2'),\
       substring_index(col('features_85.indexes'),',',2).cast('double').alias('features_85_indexes_4'),\
       substring_index(col('features_85.indexes'),',',3).cast('double').alias('features_85_indexes_4'),\
       substring_index(col('features_85.indexes'),',',4).cast('double').alias('features_85_indexes_5'),\
       substring_index(col('features_85.indexes'),',',5).cast('double').alias('features_85_indexes_6'),\
       substring_index(col('features_85.indexes'),',',6).cast('double').alias('features_85_indexes_7'),\
       substring_index(col('features_85.indexes'),',',7).cast('double').alias('features_85_indexes_8'),\
       substring_index(col('features_85.indexes'),',',8).cast('double').alias('features_85_indexes_9'),\
       substring_index(col('features_85.indexes'),',',9).cast('double').alias('features_85_indexes_10'),\
       substring_index(col('features_85.indexes'),',',10).cast('double').alias('features_85_indexes_11'),\
       substring_index(col('features_85.indexes'),',',11).cast('double').alias('features_85_indexes_12'),\
       substring_index(col('features_85.indexes'),',',12).cast('double').alias('features_85_indexes_13'),\
       substring_index(col('features_85.indexes'),',',13).cast('double').alias('features_85_indexes_14'),\
       substring_index(col('features_85.indexes'),',',14).cast('double').alias('features_85_indexes_15'),\
       substring_index(col('features_85.indexes'),',',15).cast('double').alias('features_85_indexes_16'),\
       substring_index(col('features_85.indexes'),',',16).cast('double').alias('features_85_indexes_17'),\
       substring_index(col('features_85.indexes'),',',17).cast('double').alias('features_85_indexes_18'),\
       substring_index(col('features_85.indexes'),',',18).cast('double').alias('features_85_indexes_19'),\
       substring_index(col('features_85.indexes'),',',19).cast('double').alias('features_85_indexes_20'),\
       substring_index(col('features_85.indexes'),',',20).cast('double').alias('features_85_indexes_21'),\
       substring_index(col('features_85.indexes'),',',21).cast('double').alias('features_85_indexes_22'),\
       substring_index(col('features_85.indexes'),',',22).cast('double').alias('features_85_indexes_23'),\
       substring_index(col('features_85.indexes'),',',23).cast('double').alias('features_85_indexes_24'),\
       substring_index(col('features_85.indexes'),',',24).cast('double').alias('features_85_indexes_25'),\
       substring_index(col('features_85.indexes'),',',25).cast('double').alias('features_85_indexes_26'),\
       substring_index(col('features_85.indexes'),',',26).cast('double').alias('features_85_indexes_27'),\
       substring_index(col('features_85.indexes'),',',27).cast('double').alias('features_85_indexes_28'),\
       substring_index(col('features_85.indexes'),',',28).cast('double').alias('features_85_indexes_29'),\
       substring_index(col('features_85.indexes'),',',29).cast('double').alias('features_85_indexes_30'),\
       substring_index(col('features_85.indexes'),',',30).cast('double').alias('features_85_indexes_31'),\
       substring_index(col('features_85.indexes'),',',31).cast('double').alias('features_85_indexes_32'),\
       substring_index(col('features_85.indexes'),',',32).cast('double').alias('features_85_indexes_33'),\
       substring_index(col('features_85.indexes'),',',33).cast('double').alias('features_85_indexes_34'),\
       substring_index(col('features_85.indexes'),',',34).cast('double').alias('features_85_indexes_35'),\
       substring_index(col('features_85.indexes'),',',35).cast('double').alias('features_85_indexes_36'),\
       substring_index(col('features_85.indexes'),',',36).cast('double').alias('features_85_indexes_37'),\
       substring_index(col('features_85.indexes'),',',37).cast('double').alias('features_85_indexes_38'),\
       substring_index(col('features_85.indexes'),',',38).cast('double').alias('features_85_indexes_39'),\
       substring_index(col('features_85.indexes'),',',39).cast('double').alias('features_85_indexes_40'),\
       substring_index(col('features_85.indexes'),',',40).cast('double').alias('features_85_indexes_41'),\
       substring_index(col('features_85.indexes'),',',41).cast('double').alias('features_85_indexes_42'),\
       substring_index(col('features_85.indexes'),',',42).cast('double').alias('features_85_indexes_43'),\
       substring_index(col('features_85.indexes'),',',43).cast('double').alias('features_85_indexes_44'),\
       substring_index(col('features_85.indexes'),',',44).cast('double').alias('features_85_indexes_45'),\
       substring_index(col('features_85.indexes'),',',45).cast('double').alias('features_85_indexes_46'),\
       substring_index(col('features_85.indexes'),',',46).cast('double').alias('features_85_indexes_47'),\
       substring_index(col('features_85.indexes'),',',47).cast('double').alias('features_85_indexes_48'),\
       substring_index(col('features_85.indexes'),',',48).cast('double').alias('features_85_indexes_49'),\
       substring_index(col('features_85.indexes'),',',49).cast('double').alias('features_85_indexes_50'),\
       substring_index(col('features_85.indexes'),',',50).cast('double').alias('features_85_indexes_51'),\
       substring_index(col('features_85.indexes'),',',51).cast('double').alias('features_85_indexes_52'),\
       substring_index(col('features_85.indexes'),',',52).cast('double').alias('features_85_indexes_53'),\
       substring_index(col('features_85.indexes'),',',53).cast('double').alias('features_85_indexes_54'),\
       substring_index(col('features_85.indexes'),',',54).cast('double').alias('features_85_indexes_55'),\
       substring_index(col('features_85.indexes'),',',55).cast('double').alias('features_85_indexes_56'),\
       substring_index(col('features_85.indexes'),',',56).cast('double').alias('features_85_indexes_57'),\
       substring_index(col('features_85.indexes'),',',57).cast('double').alias('features_85_indexes_58'),\
       substring_index(col('features_85.indexes'),',',58).cast('double').alias('features_85_indexes_59'),\
       substring_index(col('features_85.indexes'),',',59).cast('double').alias('features_85_indexes_60'),\
       substring_index(col('features_85.indexes'),',',60).cast('double').alias('features_85_indexes_61'),\
       substring_index(col('features_85.indexes'),',',61).cast('double').alias('features_85_indexes_62'),\
       substring_index(col('features_85.indexes'),',',62).cast('double').alias('features_85_indexes_63'),\
       substring_index(col('features_85.indexes'),',',63).cast('double').alias('features_85_indexes_64'),\
       substring_index(col('features_85.indexes'),',',64).cast('double').alias('features_85_indexes_65'),\
       substring_index(col('features_85.indexes'),',',65).cast('double').alias('features_85_indexes_66'),\
       substring_index(col('features_85.indexes'),',',66).cast('double').alias('features_85_indexes_67'),\
       substring_index(col('features_85.indexes'),',',67).cast('double').alias('features_85_indexes_68'),\
       substring_index(col('features_85.indexes'),',',68).cast('double').alias('features_85_indexes_69'),\
       substring_index(col('features_85.indexes'),',',69).cast('double').alias('features_85_indexes_70'),\
       substring_index(col('features_85.indexes'),',',70).cast('double').alias('features_85_indexes_71'),\
       substring_index(col('features_85.indexes'),',',71).cast('double').alias('features_85_indexes_72'),\
       substring_index(col('features_85.indexes'),',',72).cast('double').alias('features_85_indexes_73'),\
       substring_index(col('features_85.indexes'),',',73).cast('double').alias('features_85_indexes_74'),\
       substring_index(col('features_85.indexes'),',',74).cast('double').alias('features_85_indexes_75'),\
       substring_index(col('features_85.indexes'),',',75).cast('double').alias('features_85_indexes_76'),\
       substring_index(col('features_85.indexes'),',',76).cast('double').alias('features_85_indexes_77'),\
       substring_index(col('features_85.indexes'),',',77).cast('double').alias('features_85_indexes_78'),\
       substring_index(col('features_85.indexes'),',',78).cast('double').alias('features_85_indexes_70'),\
       substring_index(col('features_85.indexes'),',',79).cast('double').alias('features_85_indexes_80'),\
       substring_index(col('features_85.indexes'),',',80).cast('double').alias('features_85_indexes_82'),\
       substring_index(col('features_85.indexes'),',',81).cast('double').alias('features_85_indexes_82'),\
       substring_index(col('features_85.indexes'),',',82).cast('double').alias('features_85_indexes_83'),\
       substring_index(col('features_85.indexes'),',',83).cast('double').alias('features_85_indexes_84'),\
       substring_index(col('features_85.indexes'),',',84).cast('double').alias('features_85_indexes_85'),\
       substring_index(col('features_85.values'),',',0).cast('double').alias('features_85_values_1'),\
       substring_index(col('features_85.values'),',',0).cast('double').alias('features_85_values_1'),\
       substring_index(col('features_85.values'),',',1).cast('double').alias('features_85_values_2'),\
       substring_index(col('features_85.values'),',',2).cast('double').alias('features_85_values_4'),\
       substring_index(col('features_85.values'),',',3).cast('double').alias('features_85_values_4'),\
       substring_index(col('features_85.values'),',',4).cast('double').alias('features_85_values_5'),\
       substring_index(col('features_85.values'),',',5).cast('double').alias('features_85_values_6'),\
       substring_index(col('features_85.values'),',',6).cast('double').alias('features_85_values_7'),\
       substring_index(col('features_85.values'),',',7).cast('double').alias('features_85_values_8'),\
       substring_index(col('features_85.values'),',',8).cast('double').alias('features_85_values_9'),\
       substring_index(col('features_85.values'),',',9).cast('double').alias('features_85_values_10'),\
       substring_index(col('features_85.values'),',',10).cast('double').alias('features_85_values_11'),\
       substring_index(col('features_85.values'),',',11).cast('double').alias('features_85_values_12'),\
       substring_index(col('features_85.values'),',',12).cast('double').alias('features_85_values_13'),\
       substring_index(col('features_85.values'),',',13).cast('double').alias('features_85_values_14'),\
       substring_index(col('features_85.values'),',',14).cast('double').alias('features_85_values_15'),\
       substring_index(col('features_85.values'),',',15).cast('double').alias('features_85_values_16'),\
       substring_index(col('features_85.values'),',',16).cast('double').alias('features_85_values_17'),\
       substring_index(col('features_85.values'),',',17).cast('double').alias('features_85_values_18'),\
       substring_index(col('features_85.values'),',',18).cast('double').alias('features_85_values_19'),\
       substring_index(col('features_85.values'),',',19).cast('double').alias('features_85_values_20'),\
       substring_index(col('features_85.values'),',',20).cast('double').alias('features_85_values_21'),\
       substring_index(col('features_85.values'),',',21).cast('double').alias('features_85_values_22'),\
       substring_index(col('features_85.values'),',',22).cast('double').alias('features_85_values_23'),\
       substring_index(col('features_85.values'),',',23).cast('double').alias('features_85_values_24'),\
       substring_index(col('features_85.values'),',',24).cast('double').alias('features_85_values_25'),\
       substring_index(col('features_85.values'),',',25).cast('double').alias('features_85_values_26'),\
       substring_index(col('features_85.values'),',',26).cast('double').alias('features_85_values_27'),\
       substring_index(col('features_85.values'),',',27).cast('double').alias('features_85_values_28'),\
       substring_index(col('features_85.values'),',',28).cast('double').alias('features_85_values_29'),\
       substring_index(col('features_85.values'),',',29).cast('double').alias('features_85_values_30'),\
       substring_index(col('features_85.values'),',',30).cast('double').alias('features_85_values_31'),\
       substring_index(col('features_85.values'),',',31).cast('double').alias('features_85_values_32'),\
       substring_index(col('features_85.values'),',',32).cast('double').alias('features_85_values_33'),\
       substring_index(col('features_85.values'),',',33).cast('double').alias('features_85_values_34'),\
       substring_index(col('features_85.values'),',',34).cast('double').alias('features_85_values_35'),\
       substring_index(col('features_85.values'),',',35).cast('double').alias('features_85_values_36'),\
       substring_index(col('features_85.values'),',',36).cast('double').alias('features_85_values_37'),\
       substring_index(col('features_85.values'),',',37).cast('double').alias('features_85_values_38'),\
       substring_index(col('features_85.values'),',',38).cast('double').alias('features_85_values_39'),\
       substring_index(col('features_85.values'),',',39).cast('double').alias('features_85_values_40'),\
       substring_index(col('features_85.values'),',',40).cast('double').alias('features_85_values_41'),\
       substring_index(col('features_85.values'),',',41).cast('double').alias('features_85_values_42'),\
       substring_index(col('features_85.values'),',',42).cast('double').alias('features_85_values_43'),\
       substring_index(col('features_85.values'),',',43).cast('double').alias('features_85_values_44'),\
       substring_index(col('features_85.values'),',',44).cast('double').alias('features_85_values_45'),\
       substring_index(col('features_85.values'),',',45).cast('double').alias('features_85_values_46'),\
       substring_index(col('features_85.values'),',',46).cast('double').alias('features_85_values_47'),\
       substring_index(col('features_85.values'),',',47).cast('double').alias('features_85_values_48'),\
       substring_index(col('features_85.values'),',',48).cast('double').alias('features_85_values_49'),\
       substring_index(col('features_85.values'),',',49).cast('double').alias('features_85_values_50'),\
       substring_index(col('features_85.values'),',',50).cast('double').alias('features_85_values_51'),\
       substring_index(col('features_85.values'),',',51).cast('double').alias('features_85_values_52'),\
       substring_index(col('features_85.values'),',',52).cast('double').alias('features_85_values_53'),\
       substring_index(col('features_85.values'),',',53).cast('double').alias('features_85_values_54'),\
       substring_index(col('features_85.values'),',',54).cast('double').alias('features_85_values_55'),\
       substring_index(col('features_85.values'),',',55).cast('double').alias('features_85_values_56'),\
       substring_index(col('features_85.values'),',',56).cast('double').alias('features_85_values_57'),\
       substring_index(col('features_85.values'),',',57).cast('double').alias('features_85_values_58'),\
       substring_index(col('features_85.values'),',',58).cast('double').alias('features_85_values_59'),\
       substring_index(col('features_85.values'),',',59).cast('double').alias('features_85_values_60'),\
       substring_index(col('features_85.values'),',',60).cast('double').alias('features_85_values_61'),\
       substring_index(col('features_85.values'),',',61).cast('double').alias('features_85_values_62'),\
       substring_index(col('features_85.values'),',',62).cast('double').alias('features_85_values_63'),\
       substring_index(col('features_85.values'),',',63).cast('double').alias('features_85_values_64'),\
       substring_index(col('features_85.values'),',',64).cast('double').alias('features_85_values_65'),\
       substring_index(col('features_85.values'),',',65).cast('double').alias('features_85_values_66'),\
       substring_index(col('features_85.values'),',',66).cast('double').alias('features_85_values_67'),\
       substring_index(col('features_85.values'),',',67).cast('double').alias('features_85_values_68'),\
       substring_index(col('features_85.values'),',',68).cast('double').alias('features_85_values_69'),\
       substring_index(col('features_85.values'),',',69).cast('double').alias('features_85_values_70'),\
       substring_index(col('features_85.values'),',',70).cast('double').alias('features_85_values_71'),\
       substring_index(col('features_85.values'),',',71).cast('double').alias('features_85_values_72'),\
       substring_index(col('features_85.values'),',',72).cast('double').alias('features_85_values_73'),\
       substring_index(col('features_85.values'),',',73).cast('double').alias('features_85_values_74'),\
       substring_index(col('features_85.values'),',',74).cast('double').alias('features_85_values_75'),\
       substring_index(col('features_85.values'),',',75).cast('double').alias('features_85_values_76'),\
       substring_index(col('features_85.values'),',',76).cast('double').alias('features_85_values_77'),\
       substring_index(col('features_85.values'),',',77).cast('double').alias('features_85_values_78'),\
       substring_index(col('features_85.values'),',',78).cast('double').alias('features_85_values_70'),\
       substring_index(col('features_85.values'),',',79).cast('double').alias('features_85_values_80'),\
       substring_index(col('features_85.values'),',',80).cast('double').alias('features_85_values_82'),\
       substring_index(col('features_85.values'),',',81).cast('double').alias('features_85_values_82'),\
       substring_index(col('features_85.values'),',',82).cast('double').alias('features_85_values_83'),\
       substring_index(col('features_85.values'),',',83).cast('double').alias('features_85_values_84'),\
       substring_index(col('features_85.values'),',',84).cast('double').alias('features_85_values_85'),\
       col('ngramscounts_85.size').cast('int'), col('ngramscounts_85.type').cast('int'),\
       substring_index(col('ngramscounts_85.indexes'),',',0).cast('double').alias('ngramscounts_85_indexes_1'),\
       substring_index(col('ngramscounts_85.indexes'),',',1).cast('double').alias('ngramscounts_85_indexes_2'),\
       substring_index(col('ngramscounts_85.indexes'),',',2).cast('double').alias('ngramscounts_85_indexes_3'),\
       substring_index(col('ngramscounts_85.indexes'),',',3).cast('double').alias('ngramscounts_85_indexes_4'),\
       substring_index(col('ngramscounts_85.indexes'),',',4).cast('double').alias('ngramscounts_85_indexes_5'),\
       substring_index(col('ngramscounts_85.indexes'),',',5).cast('double').alias('ngramscounts_85_indexes_6'),\
       substring_index(col('ngramscounts_85.indexes'),',',6).cast('double').alias('ngramscounts_85_indexes_7'),\
       substring_index(col('ngramscounts_85.indexes'),',',7).cast('double').alias('ngramscounts_85_indexes_8'),\
       substring_index(col('ngramscounts_85.values'),',',0).cast('double').alias('ngramscounts_85values_1'),\
       substring_index(col('ngramscounts_85.values'),',',1).cast('double').alias('ngramscounts_85values_2'),\
       substring_index(col('ngramscounts_85.values'),',',2).cast('double').alias('ngramscounts_85values_3'),\
       substring_index(col('ngramscounts_85.values'),',',3).cast('double').alias('ngramscounts_85values_4'),\
       substring_index(col('ngramscounts_85.values'),',',4).cast('double').alias('ngramscounts_85values_5'),\
       substring_index(col('ngramscounts_85.values'),',',5).cast('double').alias('ngramscounts_85values_6'),\
       substring_index(col('ngramscounts_85.values'),',',6).cast('double').alias('ngramscounts_85values_7'),\
       substring_index(col('ngramscounts_85.values'),',',7).cast('double').alias('ngramscounts_85values_8'))\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
# Train just with the first 25000 random hash_message
#
not_fraud_label_train_pd=not_fraud_label_read_df.toPandas().head(25000)  
not_fraud_label_test_pd=not_fraud_label_train_pd.toPandas().head(500)
not_fraud_label_train=h2o.H2OFrame(not_fraud_label_train_pd)
not_fraud_label_test=h2o.H2OFrame(not_fraud_label_test_pd)
#
#
###### TRAINING PROCESS ############
# RBIND "Merge" all of 3 internal ccrawl and phishtank with rbind .
# function merge() doesn't work if both H2O/dataframes have same variables
#
train = fraud_label_train.rbind(not_fraud_label_train)
test = fraud_label_test.rbind(not_fraud_label_test)
#
#
print("train")
print(train.head(10))
print("test")
print(test.head(10))
#
# Identify predictors and response
x = train.columns
#
# Fraud Label to be learned in the model from the atrributes of the ngram85 learned words
#
y= 'fraud_label'
x.remove(y)
#
# For binary classification, response should be a factor
#train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()
#
# http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# Balance Classes to compensate unbalanced data
# Run AutoML for 25 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=25, seed=19, exclude_algos=["DRF","GLM"])
aml.train(x=x, y=y, training_frame=train)
#
sc.stop()
#
print("Model NGrams Vectors Data Done!")
#

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 20 hours 32 mins
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,2 months and 29 days
H2O cluster name:,H2O_from_python_siemanalyst_f03p4d
H2O cluster total nodes:,1
H2O cluster free memory:,4.908 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


Value UDF : standard_fraud_ngram=
865fc00f8a94dd1c467c39753bfb2b18626f2827db85a05654caf048e657b6856afad3fef87873197e7f92e0ff760c8f04cd5484c910a037476643addfdb5c31
root
 |-- features_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- fraud_label: long (nullable = true)
 |-- hash_message: string (nullable = true)
 |-- ngrams_85: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngramscounts_85: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- words: array (nullable = true)

In [69]:
sc.stop()